In [1]:
import pandas as pd
import copy

In [2]:
file_name = 'SB_task_data.csv'

In [3]:
# Читаем файл. Структура файла соответствует структуре, которую рисовали на встрече.
# Считаем, что поле ID (childID) - уникально (то есть элемент имеет только одного родителя)
# Считаем, что иерархия в файле корректна (нет циклов)
hierarchy_list = list()
with open(file_name, 'r') as hier_file:
    for i, line in enumerate(hier_file):        
        if i != 0:
            hierarchy_list.append(line.strip().split(';'))

In [4]:
# Делаем еще список working area
# Копию делаем через deepcopy (если делать copy, то будет беда :) не сразу это понял, пришлось повозиться)
h_wa = copy.deepcopy(hierarchy_list)
# Данные в файлике сделаны в произвольном порядке (чтобы проверить, что сортировка работает)
# Выводим на печать сортированный список.
sorted(hierarchy_list, key = lambda x: x[0])

[['01000000', ''],
 ['01010000', '01000000'],
 ['01010100', '01010000'],
 ['01010200', '01010000'],
 ['01020000', '01000000'],
 ['02000000', ''],
 ['02010000', '02000000'],
 ['02010100', '02010000'],
 ['02010101', '02010100'],
 ['03000000', ''],
 ['03010000', '03000000'],
 ['04000000', '']]

**Задачу понимаем следующим образом:**

Дана иерархия в виде списка пар [childID, parentID]. 
Необходимо распечать эту иерархия в понятном для человека виде (как в bex-отчетах).

**Решение следующее:**

Хотим получить список, который можно вывести на экран "в лоб". 
Этот список будет иметь следующую структуру: [[elem1, elem_level], [elem2, elem_level], ...]. Где elem - это ID элемента иерархии, а elem_level - уровень элемента в иерархии(он определяет отступ при выводе). Элементы этого списка должны стоять в правильном порядке (в порядке, в котором будем выводить на печать).

Соответственно задача сводится к:
1. Определению уровня элемента в иерархии
2. Определению положения элемента в иерархии (какой порядковый номер у элемента при печати)


In [5]:
# В этой ячейке определены все функции

# Функия возвращает родителя произвольного ID
def get_parentID(node, hierarchy):
    for elem in hierarchy:
        if elem[0] == node:
            return elem[1]
        
# Удаляем из иерархии все элементы, у которых нет родителя         
def del_curr_level_elems(hierarchy, curr_level_elems):
    for elem in curr_level_elems:
        hierarchy.remove([elem,''])

# Заменяем родителя elem на '' (то есть возводим все элементы с родителем = elem до нулевого уровня иерархии)
def del_par_cur_elem(h_wa, elem):
    for i, hier_elem in enumerate(h_wa):
            if hier_elem[1] == elem:
                hier_elem[1] = ''

# вставляем в список(hierarchy_parsed) пару (node, level). 
# Вставку делаем "правильно" то есть под элемент parentID 
def insert_node(hierarchy_parsed, node, parentID, level):
    for i, elem in enumerate(hierarchy_parsed):
        if elem[0] == parentID:
            hierarchy_parsed.insert(i+1,[node,level])

# Находим все элементы первого(наивысшего) уровня в иерархии h_wa
def get_curr_level_nodes(h_wa, level):
    curr_level_elems = list()
    for elem in h_wa:
        if elem[1] == '':
            curr_level_elems.append(elem[0])
    return sorted(curr_level_elems, reverse = (level != 0))

In [6]:
# Создаем целевой список, тот который будем печатать
hierarchy_parsed = list()

level = 0

#Делаем "бесконечный" цикл, пока все элементы из h_wa(начальной иерархии) не будут перенесены в список для печати hierarchy_parsed. 
#То есть в итоге список h_wa будет пустым. А hierarchy_parsed будет содержать все элементы иерархии.

#По сути цикл идет по уровням иерархии (от наивысшего)
#На каждой итерации мы будем определять все элементы текущего уровня иерархии.

while True:

#Из цикла выходим когда все элементы включены в список для печати hierarchy_parsed
    if len(h_wa) == 0:
        break

#Находим все элементы, у которых нет родителя - их и будем включать в hierarchy_parsed      
    curr_level_nodes = get_curr_level_nodes(h_wa, level)

#Итерируемся по этим элементам    
    for i, elem in enumerate(curr_level_nodes):
#Вставляем элементы из curr_level_nodes в hierarchy_parsed в нужно порядке
#То есть с индексом i+1, где i - индекс родителя
#Элементы высшего уровня иерархии просто вставляем (сортировка уже сделана)
        if level == 0:
            hierarchy_parsed.insert(i, [elem, level])
#Если node имеет родителя, то для него определяем parentID и вставляем элемент в hierarchy_parsed 
        else:
            parentID = get_parentID(elem, hierarchy_list)
            insert_node(hierarchy_parsed, elem, parentID, level)
#Теперь удаляем элемент, который мы выше вставили в hierarchy_parsed 
        try:
            h_wa.remove([elem, ''])
        except ValueError:
            pass
#Теперь удаляем родителя для всех childID с родителем из curr_level_nodes
#На следующей итерации мы будем 
        del_par_cur_elem(h_wa, elem)

    level += 1

In [7]:
#Выводим список на печать
for elem in hierarchy_parsed:
    print('\t'*elem[1], elem[0])

 01000000
	 01010000
		 01010100
		 01010200
	 01020000
 02000000
	 02010000
		 02010100
			 02010101
 03000000
	 03010000
 04000000
